In [49]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras.backend as K
from tensorflow import keras
from keras.layers import Reshape, Conv2DTranspose, PReLU
from keras.utils import to_categorical, plot_model
from keras.layers import Concatenate
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.image import resize
from scipy.linalg import sqrtm
import tensorflow as tf
from IPython.display import clear_output, HTML
import glob
from keras.layers import AveragePooling2D, ZeroPadding2D, BatchNormalization, Activation, MaxPool2D, Add
from keras.layers import Normalization, Dense, Conv2D, Dropout, BatchNormalization, ReLU
from keras.models import Sequential, Model
from keras import Input
from keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU, GlobalMaxPooling2D, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import numpy as np
%pip install scikit-image
from skimage.transform import AffineTransform, warp
from keras import Sequential, Model
from keras.layers import Dense, Conv2D, Reshape, UpSampling2D, \
    BatchNormalization, Activation, Input, LeakyReLU, ZeroPadding2D, Dropout, Flatten, Conv2DTranspose
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import rotate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation, Input, LeakyReLU
from keras.initializers import RandomNormal
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
#import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import time
# import cv2
# from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout, Embedding, Concatenate


Note: you may need to restart the kernel to use updated packages.


In [50]:
# List physical GPUs and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [51]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('emnist-letters-train.csv', delimiter=',', header=None)

In [52]:
df = df[df[0] != -1]

# 51:24 Harry Potter Deathy Hallows 1

In [ ]:
# Define the mapping dictionary
mapping = {1: 0, 
           2: 1, 
           3: 2, 
           4: 3, 
           5: 4, 
           6: 5, 
           7: 6, 
           8: 7, 
           9: 8, 
           10: 9, 
           11: 10, 
           12: 11, 
           13: 12, 
           14: 13, 
           15: 14, 
           16: 15, 
           17: 16, 
           18: 17, 
           19: 18, 
           20: 19, 
           21: 20, 
           22: 21, 
           23: 22, 
           24: 23, 
           25: 24, 
           26: 25, 
           27: 26}

        # Map the labels column to its corresponding value
df[0] = df[0].map(mapping)

In [54]:
y_pre = np.array(df.iloc[:,0].values)
y_pre = pd.Categorical(y_pre)
X = np.array(df.iloc[:,1:].values)
X = X.reshape(-1,28,28,1)
preprocessed = []
for image in X:
    rotated_image = rotate(image, 90, reshape=False)
    flipped_image = np.flipud(rotated_image)
    preprocessed.append(flipped_image)
X_pre = np.array(preprocessed)
X = X_pre
X = X.astype('float32')
X_pre = (X - 127.5) / 127.5


In [ ]:
print(f'y_pre\n{y_pre.unique()}')

[22, 6, 15, 14, 16, ..., 19, 8, 5, 11, 0]
Length: 26
Categories (26, int64): [0, 1, 2, 3, ..., 22, 23, 24, 25]

In [55]:
def train(self, images, labels_input, epochs=10000, batch_size=256, save_interval=500, gen_steps=1):
        os.makedirs('ACGAN_Generated_Images', exist_ok=True)
        X_train = images
        y_train = labels_input

        batches_per_epoch = X_train.shape[0] // batch_size

        for epoch in range(epochs):
            for batch in range(batches_per_epoch):
                # Train Discriminator
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                labels_real = np.ones((batch_size, 1))  # Real labels

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
                gen_imgs = self.generator.predict([noise, gen_labels])
                labels_fake = np.zeros((batch_size, 1))  # Fake labels

                d_loss_real = self.discriminator.train_on_batch(imgs, [labels_real, y_train[idx]])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [labels_fake, gen_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # Train Generator
                g_loss = None
                for _ in range(gen_steps):
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
                    valid_y = np.ones((batch_size, 1))
                    g_loss = self.acgan.train_on_batch([noise, gen_labels], [valid_y, gen_labels])

                # Print the progress
                print(f"Epoch {epoch + 1}/{epochs}, Batch {batch + 1}/{batches_per_epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[3]:.2f}%] [G loss: {g_loss[0]}]")

            # Save generated images at save intervals
            if (epoch + 1) % save_interval == 0:
                self.save_images(epoch+1)

In [56]:
# class CGAN():

#     def __init__(self, rows, cols, channels, z = 100):
#         # Input shape
#         self.img_rows = rows
#         self.img_cols = cols
#         self.channels = channels
#         self.img_shape = (self.img_rows, self.img_cols, self.channels)
#         self.latent_dim = z
#         self.classes = 25
#         optimizer = Adam(0.00002, 0.5)
#         # Build and compile the discriminator
#         self.discriminator = self.define_discriminator(self.img_shape,self.classes)
#         self.discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    
#         # Build the generator
#         self.g_model = self.define_generator(self.latent_dim,self.classes)
#         # The generator takes noise as input and generates imgs
#         z = Input(shape=(self.latent_dim,))
#         img = self.g_model(z)
#         # For the combined model we will only train the generator
#         self.discriminator.trainable = False
#         # The discriminator takes generated images as input and
#         # determines validity
#         valid = self.discriminator(img)
#         # The combined model (stacked generator and discriminator)
#         # Trains the generator to fool the discriminator
#         self.combined = Model(z, valid)
#         self.combined.compile(loss='binary_crossentropy',optimizer=optimizer)

#     # define the standalone discriminator model
#     def define_discriminator(self, in_shape, n_classes):
#         in_label = Input(shape=(1,))
#         li = Embedding(n_classes, 50)(in_label)
#         n_nodes = in_shape[0] * in_shape[1]
#         li = Dense(n_nodes)(li)
#         li = Reshape((in_shape[0], in_shape[1], 1))(li)
#         in_image = Input(shape=in_shape)
#         merge = Concatenate()([in_image, li])
#         fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
#         fe = LeakyReLU(alpha=0.2)(fe)
#         fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
#         fe = LeakyReLU(alpha=0.2)(fe)
#         fe = Flatten()(fe)
#         fe = Dropout(0.4)(fe)
#         out_layer = Dense(1, activation='sigmoid')(fe)
#         model = Model([in_image, in_label], out_layer)
#         return model

#     # define the standalone generator model
#     def define_generator(self, latent_dim, n_classes):
#         in_label = Input(shape=(1,))
#         li = Embedding(n_classes, 50)(in_label)
#         n_nodes = 7 * 7
#         li = Dense(n_nodes)(li)
#         li = Reshape((7, 7, 1))(li)
#         in_lat = Input(shape=(latent_dim,))
#         n_nodes = 128 * 7 * 7
#         gen = Dense(n_nodes)(in_lat)
#         gen = LeakyReLU(alpha=0.2)(gen)
#         gen = Reshape((7, 7, 128))(gen)
#         merge = Concatenate()([gen, li])
#         gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(merge)
#         gen = LeakyReLU(alpha=0.2)(gen)
#         gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(gen)
#         gen = LeakyReLU(alpha=0.2)(gen)
#         out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
#         model = Model([in_lat, in_label], out_layer)
#         return model


#     def save_imgs(self, epoch):
#         r, c = 5, 5
#         noise = np.random.normal(0, 1, (r * c, self.latent_dim))
#         gen_imgs = self.generator.predict(noise)
#         gen_imgs = 0.5 * gen_imgs + 0.5
#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             for j in range(c):
#                 axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         os.makedirs('generated_mnist_CGAN', exist_ok=True)
#         fig.savefig("generated_mnist_CGAN/cgan_mnist_improved_%d.png" % epoch)
#         plt.close()

#     # train the generator and discriminator
#     def train(self, epochs, batch_size=256, save_internal = 500, gen_steps=1):
#         X_train = X_pre
#         y_train = y_pre

#         batches_per_epoch = X_train.shape[0] // batch_size

#         for epoch in range(epochs):
#             for batch in range(batches_per_epoch):
#                 # Train Discriminator
#                 idx = np.random.randint(0, X_train.shape[0], batch_size)
#                 imgs = X_train[idx]
#                 labels_real = np.ones((batch_size, 1))  # Real labels

#                 noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
#                 gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
#                 gen_imgs = self.generator.predict([noise, gen_labels])
#                 labels_fake = np.zeros((batch_size, 1))  # Fake labels

#                 d_loss_real = self.discriminator.train_on_batch(imgs, [labels_real, y_train[idx]])
#                 d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [labels_fake, gen_labels])
#                 d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

#                 # Train Generator
#                 g_loss = None
#                 for _ in range(gen_steps):
#                     noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
#                     gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
#                     valid_y = np.ones((batch_size, 1))
#                     g_loss = self.acgan.train_on_batch([noise, gen_labels], [valid_y, gen_labels])

#                 # Print the progress
#                 print(f"Epoch {epoch + 1}/{epochs}, Batch {batch + 1}/{batches_per_epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[3]:.2f}%] [G loss: {g_loss[0]}]")

#             # Save generated images at save intervals
#             if (epoch + 1) % save_internal == 0:
#                 self.save_images(epoch+1)


In [79]:
class CGAN():
    def __init__(self, rows, cols, channels, z=100, num_classes=26):
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z
        self.num_classes = num_classes
        optimizer = Adam(0.00002, 0.5)
        self.discriminator = self.define_discriminator(self.img_shape, self.num_classes)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.generator = self.define_generator(self.latent_dim, self.num_classes)
        z = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([z, label])
        self.discriminator.trainable = False
        valid = self.discriminator([img, label])
        self.combined = Model([z, label], valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def define_discriminator(self, in_shape, n_classes):
        in_label = Input(shape=(1,))
        li = Embedding(n_classes, 50)(in_label)
        n_nodes = in_shape[0] * in_shape[1]
        li = Dense(n_nodes)(li)
        li = Reshape((in_shape[0], in_shape[1], 1))(li)
        in_image = Input(shape=in_shape)
        merge = Concatenate()([in_image, li])
        fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(merge)
        fe = LeakyReLU(alpha=0.2)(fe)
        fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(fe)
        fe = LeakyReLU(alpha=0.2)(fe)
        fe = Flatten()(fe)
        fe = Dropout(0.4)(fe)
        out_layer = Dense(1, activation='sigmoid')(fe)
        model = Model([in_image, in_label], out_layer)
        return model

    def define_generator(self, latent_dim, n_classes):
        in_label = Input(shape=(1,))
        li = Embedding(n_classes, 50)(in_label)
        n_nodes = 7 * 7
        li = Dense(n_nodes)(li)
        li = Reshape((7, 7, 1))(li)
        in_lat = Input(shape=(latent_dim,))
        n_nodes = 128 * 7 * 7
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((7, 7, 128))(gen)
        merge = Concatenate()([gen, li])
        gen = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same')(merge)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same')(gen)
        gen = LeakyReLU(alpha=0.2)(gen)
        out_layer = Conv2D(1, (7, 7), activation='tanh', padding='same')(gen)
        model = Model([in_lat, in_label], out_layer)
        return model

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        sampled_labels = np.arange(0, r * c).reshape(-1, 1) % self.num_classes  # Ensure labels are within valid range
        gen_imgs = self.generator.predict([noise, sampled_labels])
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        os.makedirs('generated_mnist_CGAN', exist_ok=True)
        fig.savefig("generated_mnist_CGAN/cgan_mnist_improved_%d.png" % epoch)
        plt.close()

    def train(self, epochs=100, batch_size=256, save_interval=2, gen_steps=1):
        X_train = X_pre
        y_train = y_pre

        batches_per_epoch = X_train.shape[0] // batch_size

        for epoch in range(epochs):
            for batch in range(batches_per_epoch):
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                labels_real = np.ones((batch_size, 1))  # Real labels

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))  # Ensure valid range
                gen_imgs = self.generator.predict([noise, gen_labels])
                labels_fake = np.zeros((batch_size, 1))  # Fake labels

                d_loss_real = self.discriminator.train_on_batch([imgs, y_train[idx]], labels_real)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, gen_labels], labels_fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                g_loss = None
                for _ in range(gen_steps):
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))  # Ensure valid range
                    valid_y = np.ones((batch_size, 1))
                    g_loss = self.combined.train_on_batch([noise, gen_labels], valid_y)

                # Print the progress
                print(f"Epoch {epoch + 1}/{epochs}, Batch {batch + 1}/{batches_per_epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss}]")

            if (epoch + 1) % save_interval == 0:
                self.save_imgs(epoch + 1)


In [80]:
# Set image dimensions
img_rows, img_cols, channels = 28, 28, 1

# Instantiate and train the DCGAN
cgan = CGAN(img_rows, img_cols, channels)
cgan.train()

8/8 [==============================] - 0s 18ms/step
Epoch 1/100, Batch 1/346 [D loss: 0.694011926651001, acc.: 51.76%] [G loss: 0.6934709548950195]
8/8 [==============================] - 0s 20ms/step
Epoch 1/100, Batch 2/346 [D loss: 0.6883581578731537, acc.: 58.79%] [G loss: 0.6932325959205627]
8/8 [==============================] - 0s 18ms/step
Epoch 1/100, Batch 3/346 [D loss: 0.686428964138031, acc.: 54.88%] [G loss: 0.6930594444274902]
8/8 [==============================] - 0s 19ms/step
Epoch 1/100, Batch 4/346 [D loss: 0.6820458769798279, acc.: 52.73%] [G loss: 0.6926531791687012]
8/8 [==============================] - 0s 18ms/step
Epoch 1/100, Batch 5/346 [D loss: 0.6763851046562195, acc.: 53.91%] [G loss: 0.692227840423584]
8/8 [==============================] - 0s 18ms/step
Epoch 1/100, Batch 6/346 [D loss: 0.6749053299427032, acc.: 50.78%] [G loss: 0.6919347047805786]
8/8 [==============================] - 0s 18ms/step
Epoch 1/100, Batch 7/346 [D loss: 0.6700701415538788, acc

KeyboardInterrupt: 